In [ ]:
# CELL 1: DATA PREPROCESSING AND LOADING
# -------------------------------------------------
# This cell performs several preprocessing steps:
# 1. Loads the raw CAD dataset from Hugging Face.
# 2. Shuffles and subsets the training and test splits.
# 3. Preprocesses each batch by resizing images and tokenizing text.
# 4. Saves the preprocessed data as multiple shards to disk.
from datasets import load_dataset, load_from_disk, concatenate_datasets
import torch
from transformers import AutoTokenizer, AutoImageProcessor
import numpy as np
import gc
import os

# Config parameters for data processing
CACHE_DIR = "/Volumes/BIG-DATA/HUGGINGFACE_CACHE"
OUTPUT_DIR = "/Volumes/BIG-DATA/processed_gencad"
NUM_SHARDS = 10
BATCH_SIZE = 8
MAX_SAMPLES = 10_000

# Setup processors with pre-trained models
encoder_checkpoint = "google/vit-base-patch16-224-in21k"
decoder_checkpoint = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(decoder_checkpoint)
image_processor = AutoImageProcessor.from_pretrained(encoder_checkpoint)

# Ensure the GPT2 tokenizer has a PAD token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_data():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print("Loading raw dataset...")
    ds = load_dataset("CADCODER/GenCAD-Code", num_proc=4, cache_dir=CACHE_DIR)
    
    # Shuffle and subset the dataset for training and testing
    ds["train"] = ds["train"].shuffle(seed=42).select(range(min(MAX_SAMPLES, len(ds["train"]))))
    ds["test"] = ds["test"].shuffle(seed=42).select(range(min(5000, len(ds["test"]))))
    
    print(f"Train samples: {len(ds['train'])}, Test samples: {len(ds['test'])}")
    
    def preprocess_batch(batch):
        # Preprocess images to 224x224 and convert to FP16 format
        pixel_values = image_processor(
            batch["image"],
            return_tensors="pt",
            size={"height": 224, "width": 224}
        ).pixel_values.to(dtype=torch.float16)
        
        # Preprocess text by tokenizing with max_length and padding
        text_inputs = tokenizer(
            batch["cadquery"],
            padding="max_length",
            truncation=True,
            max_length=256,
            return_tensors="pt"
        )
        
        return {
            "pixel_values": [pv.numpy() for pv in pixel_values],
            "labels": [lb.numpy() for lb in text_inputs.input_ids],
            "attention_mask": [am.numpy() for am in text_inputs.attention_mask]
        }
    
    # Process and save dataset in shards to avoid huge files
    for split_name in ["train", "test"]:
        split = ds[split_name]
        for i in range(NUM_SHARDS):
            shard = split.shard(num_shards=NUM_SHARDS, index=i)
            shard = shard.map(
                preprocess_batch,
                batched=True,
                batch_size=BATCH_SIZE,
                remove_columns=split.column_names,
                desc=f"Processing {split_name} shard {i+1}"
            )
            shard.set_format(type="torch", columns=["pixel_values", "labels", "attention_mask"])
            shard.save_to_disk(f"{OUTPUT_DIR}/{split_name}_shard{i}")
            gc.collect()
    
    print(" Preprocessing complete!")

# Function to load all preprocessed shards and concatenate them
def load_processed_data():
    train_shards = [load_from_disk(f"{OUTPUT_DIR}/train_shard{i}") for i in range(NUM_SHARDS)]
    train_ds = concatenate_datasets(train_shards)
    
    test_shards = [load_from_disk(f"{OUTPUT_DIR}/test_shard{i}") for i in range(NUM_SHARDS)]
    test_ds = concatenate_datasets(test_shards)
    
    return train_ds, test_ds

# Run preprocessing if the output directory does not exist
if not os.path.exists(OUTPUT_DIR):
    print("Running data preprocessing...")
    preprocess_data()

# Load the preprocessed datasets and print a summary
train_ds, test_ds = load_processed_data()
print(f" Loaded - Train: {len(train_ds)}, Test: {len(test_ds)}")
print("Sample:", train_ds[0].keys())

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Running data preprocessing...
Loading raw dataset...
Train samples: 10000, Test samples: 5000


Processing train shard 1:   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Processing train shard 2:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

Processing test shard 9:   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Processing test shard 10:   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

✅ Preprocessing complete!
✅ Loaded - Train: 10000, Test: 5000
Sample: dict_keys(['pixel_values', 'labels', 'attention_mask'])


In [ ]:
# CELL 2: MODEL SETUP AND CONFIGURATION
# -------------------------------------------------
# This cell sets up the VisionEncoderDecoder model.
# Steps include:
# 1. Loading the encoder (ViT) and decoder (GPT2) from pre-trained checkpoints.
# 2. Loading the corresponding tokenizer and image processor.
# 3. Configuring the tokenizer to include a padding token if missing.
# 4. Adjusting the model configuration parameters for token IDs and generation.
# 5. Defining a simple data collator for batching during training.
# 6. Moving the model to GPU if available.
from transformers import (
    VisionEncoderDecoderModel,
    AutoTokenizer,
    AutoImageProcessor
)
import torch

# Define model checkpoints for encoder and decoder
encoder_checkpoint = "google/vit-base-patch16-224-in21k"
decoder_checkpoint = "gpt2"

# Load the VisionEncoderDecoder model from the specified checkpoints
print("Loading VisionEncoderDecoder model...")
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_checkpoint,
    decoder_checkpoint
)

# Load tokenizer and image processor
tokenizer = AutoTokenizer.from_pretrained(decoder_checkpoint)
image_processor = AutoImageProcessor.from_pretrained(encoder_checkpoint)

# Ensure the GPT2 tokenizer has a pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Configure the model's token IDs and vocabulary size
model.config.decoder_start_token_id = tokenizer.bos_token_id if tokenizer.bos_token_id else tokenizer.eos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

# Set additional generation parameters
model.config.max_length = 256
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

# Define a data collator to prepare batches with proper padding for training
def data_collator(batch):
    pixel_values = torch.stack([example["pixel_values"] for example in batch])
    labels = torch.stack([example["labels"] for example in batch])
    
    # Replace padding token ids with -100 for loss computation
    labels[labels == tokenizer.pad_token_id] = -100
    
    return {
        "pixel_values": pixel_values,
        "labels": labels
    }

print(" Model setup complete!")
print(f"Encoder: {model.encoder.__class__.__name__}")
print(f"Decoder: {model.decoder.__class__.__name__}")
print(f"Vocab size: {model.config.vocab_size}")
print(f"Model parameters: {model.num_parameters():,}")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model device: {device}")

Loading VisionEncoderDecoder model...


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['transformer.h.0.crossattention.c_attn.bias', 'transformer.h.0.crossattention.c_attn.weight', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.0.crossattention.c_proj.weight', 'transformer.h.0.crossattention.q_attn.bias', 'transformer.h.0.crossattention.q_attn.weight', 'transformer.h.0.ln_cross_attn.bias', 'transformer.h.0.ln_cross_attn.weight', 'transformer.h.1.crossattention.c_attn.bias', 'transformer.h.1.crossattention.c_attn.weight', 'transformer.h.1.crossattention.c_proj.bias', 'transformer.h.1.crossattention.c_proj.weight', 'transformer.h.1.crossattention.q_attn.bias', 'transformer.h.1.crossattention.q_attn.weight', 'transformer.h.1.ln_cross_attn.bias', 'transformer.h.1.ln_cross_attn.weight', 'transformer.h.10.crossattention.c_attn.bias', 'transformer.h.10.crossattention.c_attn.weight', 'transformer.h.10.crossattention.c_proj.bias', 'transformer.h.10.cros

✅ Model setup complete!
Encoder: ViTModel
Decoder: GPT2LMHeadModel
Vocab size: 50257
Model parameters: 239,195,904
Model device: cpu


In [ ]:
# CELL 3: MODEL TRAINING AND EVALUATION
# -------------------------------------------------
# This cell trains the model and evaluates its performance.
# Steps include:
# 1. Defining the training arguments (hyperparameters, logging, saving strategies).
# 2. Initializing the Seq2SeqTrainer with the training and evaluation datasets.
# 3. Training the model with error handling and saving checkpoints.
# 4. Running a simple inference test post training.
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
import torch

# Set up training hyperparameters and arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./baseline_model",
    eval_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=4,  # Low batch size to conserve memory
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # Effective batch size becomes 8
    num_train_epochs=1,  # Use one epoch for initial testing
    save_total_limit=1,
    predict_with_generate=True,
    generation_max_length=128,
    logging_steps=50,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    dataloader_num_workers=0,  # Prevent multithreading issues
    dataloader_pin_memory=False,
    remove_unused_columns=True,
    fp16=False,  # Disable FP16 to avoid precision errors
    report_to=None,
    push_to_hub=False
)

print("Training arguments:")
print(f"- Batch size: {training_args.per_device_train_batch_size}")
print(f"- Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"- Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"- Epochs: {training_args.num_train_epochs}")
print(f"- Output dir: {training_args.output_dir}")

# Initialize the trainer with the model, training arguments, and datasets
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds.select(range(100)),  # Use a smaller evaluation set to avoid memory issues
    processing_class=tokenizer,
    data_collator=data_collator
)

print("🚀 Starting training...")
print(f"Train samples: {len(train_ds)}")
print(f"Eval samples: 100")

# Train the model with error handling
try:
    # Clear CUDA cache before training if available
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    trainer.train()
    print("✅ Training completed successfully!")
    
    # Save the final trained model and tokenizer
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)
    
    print(f"💾 Model saved to {training_args.output_dir}")
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    print("Trying to save current state...")
    try:
        trainer.save_model("./emergency_save")
        print("Emergency save completed")
    except:
        print("Emergency save also failed")

# Simple inference test to validate model generation
print("\n🔍 Testing inference...")
try:
    sample = test_ds[0]
    pixel_values = sample["pixel_values"].unsqueeze(0).to(model.device)
    
    model.eval()
    with torch.no_grad():
        generated_ids = model.generate(
            pixel_values=pixel_values,
            max_length=64,
            num_beams=2,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(f"Generated: {generated_text[:200]}...")
    
except Exception as e:
    print(f"Inference failed: {e}")

print("✅ Script completed!")

# Memory cleanup to free GPU and system memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
# Inference Cell: Generate and compare CadQuery code
# -------------------------------------------------
# Steps:
# 1. Load the trained model, tokenizer, and image processor from a checkpoint.
# 2. Load a preprocessed test shard.
# 3. Select a sample from the dataset.
# 4. Generate predicted CadQuery code using the model.
# 5. Print both the predicted and ground truth CadQuery code for comparison.
import torch
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoImageProcessor
from datasets import load_from_disk

# Specify paths for checkpoint and processed test data
CHECKPOINT_DIR = "baseline_model_checkpoint-20250830T093207Z-1-001/baseline_model_checkpoint"
PROCESSED_DATA_DIR = "/Volumes/BIG-DATA/processed_gencad"  # adjust if needed

print("Loading trained model...")
model = VisionEncoderDecoderModel.from_pretrained(CHECKPOINT_DIR)
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

# Move model to GPU if available and set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device).eval()

print("Loading test dataset...")
test_ds = load_from_disk(f"{PROCESSED_DATA_DIR}/test_shard0")
print(f"Test samples: {len(test_ds)}")

# Pick one sample and prepare its pixel values for inference
sample = test_ds[0]
pixel_values = sample["pixel_values"].unsqueeze(0).to(device)

print("\n Running inference...")
with torch.no_grad():
    generated_ids = model.generate(
        pixel_values=pixel_values,
        max_length=128,
        num_beams=4,
        length_penalty=2.0,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Display the predicted and ground truth CadQuery code
print("\n=== Predicted CadQuery Code ===")
print(generated_text)

print("\n=== Ground Truth CadQuery Code ===")
print(tokenizer.decode(sample["labels"], skip_special_tokens=True))

Loading trained model...


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Loading test dataset...
Test samples: 500

🔍 Running inference...

=== Predicted CadQuery Code ===
 cad as a cad
 Gener aplane sketch for 0wpsch =qWork(qPl(qVector0000125 0006,.,.),qVector100600000001 00 00006;qVector612399367e17 -.,006 -.2335766644-).To006 00074742263,.To00742263474253line1line006line106line007line001line003line000line06line0100101line01line06ext0007ext0

=== Ground Truth CadQuery Code ===
import cadquery as cq
# Generating a workplane for sketch 0
wp_sketch0 = cq.Workplane(cq.Plane(cq.Vector(0.0, 0.0, -0.078125), cq.Vector(1.0, 6.123233995736766e-17, -6.123233995736766e-17), cq.Vector(6.123233995736766e-17, -1.0, 6.123233995736766e-17)))
loop0=wp_sketch0.moveTo(0.039473684210526314, 0.0).lineTo(0.039473684210526314, 0.05921052631578947).lineTo(0.29605263157894735, 0.05921052631578947).lineTo(0.29605263157894735, 0.0).lineTo(0.3289473684210526, 0.0).lineTo(0.3289473684210526, 0.05921052631578947


In [ ]:
# Inference Cell: Evaluate generated code using BLEU score
# -------------------------------------------------
# Steps:
# 1. Load the trained model, tokenizer, and image processor.
# 2. Load the processed test dataset.
# 3. Define a function to compute the BLEU score for a single prediction.
# 4. Run inference on a sample and compute BLEU score compared to the ground truth.
# 5. Optionally, compute the average BLEU score over multiple samples.
import torch
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoImageProcessor
from datasets import load_from_disk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Specify paths for checkpoint and processed data
CHECKPOINT_DIR = "baseline_model_checkpoint-20250830T093207Z-1-001/baseline_model_checkpoint"
PROCESSED_DATA_DIR = "/Volumes/BIG-DATA/processed_gencad"

print("Loading trained model...")
model = VisionEncoderDecoderModel.from_pretrained(CHECKPOINT_DIR)
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

# Move model to GPU if available and set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device).eval()

print("Loading test dataset...")
test_ds = load_from_disk(f"{PROCESSED_DATA_DIR}/test_shard0")
print(f"Test samples: {len(test_ds)}")

# Function to calculate the BLEU score for a single prediction vs reference text
def calculate_bleu(pred_text, ref_text):
    smoothie = SmoothingFunction().method4
    pred_tokens = pred_text.split()
    ref_tokens = ref_text.split()
    return sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothie)

# Run inference and compute BLEU score for one sample
sample = test_ds[0]
pixel_values = sample["pixel_values"].unsqueeze(0).to(device)

with torch.no_grad():
    generated_ids = model.generate(
        pixel_values=pixel_values,
        max_length=128,
        num_beams=4,
        length_penalty=2.0,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
ground_truth_text = tokenizer.decode(sample["labels"], skip_special_tokens=True)

print("\n=== Predicted CadQuery Code ===")
print(generated_text)
print("\n=== Ground Truth CadQuery Code ===")
print(ground_truth_text)

# Compute BLEU score for the sample and print it
bleu_score = calculate_bleu(generated_text, ground_truth_text)
print(f"\n BLEU score (sample 0): {bleu_score:.4f}")

# Optionally, compute the average BLEU score over N samples
N = 50  # Adjust the number of samples as needed
total_bleu = 0
for i in range(N):
    sample = test_ds[i]
    pixel_values = sample["pixel_values"].unsqueeze(0).to(device)

    with torch.no_grad():
        generated_ids = model.generate(pixel_values=pixel_values, max_length=128, num_beams=4)
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    ref = tokenizer.decode(sample["labels"], skip_special_tokens=True)

    total_bleu += calculate_bleu(pred, ref)

avg_bleu = total_bleu / N
print(f"\nAverage BLEU score over {N} samples: {avg_bleu:.4f}")

Loading trained model...


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Loading test dataset...
Test samples: 500


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== Predicted CadQuery Code ===
 cad as a cad
 Gener aplane sketch for 0wpsch =qWork(qPl(qVector0000125 0006,.,.),qVector100600000001 00 00006;qVector612399367e17 -.,006 -.2335766644-).To006 00074742263,.To00742263474253line1line006line106line007line001line003line000line06line0100101line01line06ext0007ext0

=== Ground Truth CadQuery Code ===
import cadquery as cq
# Generating a workplane for sketch 0
wp_sketch0 = cq.Workplane(cq.Plane(cq.Vector(0.0, 0.0, -0.078125), cq.Vector(1.0, 6.123233995736766e-17, -6.123233995736766e-17), cq.Vector(6.123233995736766e-17, -1.0, 6.123233995736766e-17)))
loop0=wp_sketch0.moveTo(0.039473684210526314, 0.0).lineTo(0.039473684210526314, 0.05921052631578947).lineTo(0.29605263157894735, 0.05921052631578947).lineTo(0.29605263157894735, 0.0).lineTo(0.3289473684210526, 0.0).lineTo(0.3289473684210526, 0.05921052631578947

🔵 BLEU score (sample 0): 0.0099


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati